# 16장 - 순환 신경망으로 순차 데이터 모델링 (part 1)

In [1]:
from IPython.display import Image

# 순차 데이터 소개

시퀀스 데이터 또는 **시퀀스**(sequence)로 불리는 순차 데이터의 특징에 관해 알아보면서 RNN을 알아보자.  
시퀀스에는 다른 종류의 데이터와는 구별되는 독특한 성질이 있다. 시퀀스 데이터를 표현하는 방법과 시퀀스 데이터를 위한 여러 가지 모델을 살펴보자.

## 순차 데이터 모델링: 순서를 고려한다

다른 데이터 타입과 다르게 시퀀스는 특별하다. 시퀀스 원소들은 특성 순서가 있으므로 상호 독립적이지 않기 때문이다. 일반적으로 지도 학습을 위한 머신 러닝 알고리즘은 입력 데이터가 **독립 동일 분포**(Independent and Identically Ditributed, IID)라고 가정한다. 즉, 훈련 샘플이 상호 독립(mutually indenpendence)적이고 같은 분포에 속한다는 의미이다.  
상호 독립 가정에 기반한다는 점에서 모델에 전달되는 훈련 샘플의 순서는 관계가 없다. 붓꽃 데이터셋에서 각 꽃은 개별적으로 측정되었고 한 꽃의 측정값이 다른 꽃의 측정값에 영향을 미치지 않는다.  

하지만 시퀀스를 다룰 때는 이런 가정이 유효하지 않다. 시퀀스의 정의가 순서를 고려하기 떄문이다. 특정 주식의 가격을 예측하는 것이 이런 경우에 해당한다. 예를 들어 n개의 훈련 샘플을 가지고 있다면 각 훈련 샘플을 특정한 날의 이 주식 가격을 나타낸다. 다음 3일 동안의 주식 가격을 예측하는 작업이라면 훈련 샘플을 랜덤 순서로 사용하는 것이 아니라 날짜 순서대로 정렬된 이전 주식 가격을 고려하여 트렌드를 감지하는 것이 합리적이다. 

### Note
#### 순차 데이터 vs 시계열 데이터
시계열(time series) 데이터는 순차 데이터의 특별한 한 종류이다. 각 샘플이 시간 차원에 연관되어 있다. 시계열 데이터에서는 연속적인 타임스탬프를 따라 샘플을 얻는다. 따라서 시간 차원이 데이터 포인트 사이의 순서를 결정한다. 예를 들어 주식 가격과 녹화된 음성이나 대화가 시계열 데이터이다.  

모든 순차 데이터가 시간 차원을 가지는 것은 아니다. 예를 들어 텍스트 데이터나 DNA 시퀀스는 샘플이 순서를 가지지만 시계열 데이터로 볼 수 없다. 이 장에서 보겠지만 자연어 처리(Natural Language Preprocessing, NLP)와 테스트 모델링의 샘플은 시계열 데이터가 아니다. 하지만 RNN은 시계열 데이터에 사용할 수 있다.

## 시퀀스 표현

순차 데이터에서 데이터 포인트 사이의 순서가 중요하다는 것을 이해했다. 따라서 머신 러닝 모델에서 이런 순서 정보를 사용할 수 있는 방법을 찾아야 한다.  
이 장에서 시퀀스를 $<x^{(1)}, x^{(2)}, \cdots, x^{(T)}>$로 표현한다. 위 첨자는 샘플 순서를 나타낸다. $T$는 시퀀스 길이이다. 시퀀스의 좋은 예는 시계열 데이터이다. 여기서 각 샘플 포인트 $x^{(t)}$는 특정 시간 t에 속한다. 

In [2]:
Image(url='https://git.io/JLdVm', width=700)

다층 퍼셉트론(MLP)이나 이미지 데이터를 위한 CNN과 같이 지금까지 다루었던 일반 신경망 모델은 훈련 샘플이 서로 독립적이어서 순서 정보와 연관이 없다고 가정한다.  
이런 모델은 이전에 본 훈련 샘플을 기억하는 메모리가 없다고 말한다. 예를 들어 샘플이 정방향 계산과 역전파 단계를 통과하면 가중치는 훈련 샘플의 처리 순서에 상관없이 독립적으로 업데이트된다.  

이와 대조적으로 RNN은 시퀀스 모델링을 위해 고안되었으며 과거 정보를 기억하고 이에 맞추어 새로운 샘플을 처리할 수 있기 때문에 시퀀스 데이터를 다룰 때 장점을 가진다.

## 시퀀스 모델링의 종류

시퀀스 모델링에서는 언어 번역, 이미지 캡셔닝(captioning), 텍스트 생성과 같은 매력적인 애플리케이션이 많다. 하지만 적절한 구조와 방법을 찾으려면 여러 종류의 시퀀스 모델링 작업 사이의 차이점을 이해하고 구별할 수 있어야 한다.  

In [3]:
Image(url='https://git.io/JLdVO', width=700)

위의 그림에 나와 있는 입력과 출력 데이터 사이에 나타나는 여러 관계에 대해 자세히 알아보자.  
입력과 출력 데이터가 시퀀스로 표현되지 않으면 일반 데이터이므로 간단히 다층 퍼헵트론을 사용할 수 있다. 하지만 입력이나 출력 중 하나가 시퀀스라면 이런 모델링 작업은 다음 중 하나에 속할 것이다.  

- **다대일**(many-to-one): 입력 데이터가 시퀀스이지만 출력은 시퀀스가 아니고 고정 크기의 벡터나 스칼라이다. 예를 들어 감성 분석에서 입력은 텍스트(예를 들어 영화 리뷰)이고 출력은 클래스 레이블(예를 들어 리뷰어가 영화를 좋아하는지 나타내는 레이블)이다.  

- **일대다**(one-to-many): 입력 데이터가 시퀀스가 아니라 일반적인 형태이고 출력은 시퀀스이다. 이런 종류의 예로는 이미지 캡셔닝이 있다. 입력이 이미지이고 출력은 이미지 내용을 요약한 영어 문장이다.  

- **다대다**(many-to-many): 입력과 출력 배열이 모두 시퀀스이다. 이런 종류는 입력과 출력이 동기적인지에 따라 더 나눌 수 있다. 동기적인 다대다 모델링 작업의 예는 각 프레임을 레이블링하는 비디오 분류이다. 지연이 있는 다대다 모델의 예는 한 언어에서 다른 언어로 번역하는 작업이다. 예를 들어 독일어로 번역하기 전에 전체 영어 문장을 읽어 처리한다. 

# 시퀀스 모델링을 위한 RNN

## RNN 반복 구조 이해

In [4]:
Image(url='https://git.io/JLdV3', width=700)

위의 그림에서 일반적인 피드포워드 신경망과 RNN을 비교하기 위해 나란히 놓았다.  
두 네트워크 모두 하나의 은닉층만 있다. 유닛을 표시하지 않았다.  

기본 피드포워드 네트워크에서 정보는 입력에서 은닉층으로 흐른 후 은닉층에서 출력층으로 전달된다. 반면 순한 네트워크에서는 은닉층이 현재 타임 스텝(time step)의 입력층과 이전 타임 스텝의 은닉층으로부터 정보를 받는다.  

인접한 타임 스텝의 정보가 은닉층에 흐르기 때문에 네트워크가 이전 이벤트를 기억할 수 있다. 이 정보 흐름을 보통 루프로 표시한다. 그래프 표기법에서는 순환 에지(recurrent edge)라고도 하기 때문에 RNN구조 이름이 여기서 유래되었다.  

다층 퍼셉트론과 비슷하게 RNN은 여러 개의 은닉층으로 구성할 수 있다. 하나의 은닉층을 가진 RNN을 관례적으로 단일층 RNN이라고 말한다. 아달린이나 로지스틱 회귀와 같이 은닉층이 없는 단일층 신경망과 혼동하면 안된다.

In [5]:
Image(url='https://git.io/JLdVs', width=700)

위의 그림은 하나의 은닉층을 가진 RNN과 두 개의 은닉층을 가진 RNN을 보여준다.  

일반 신경망의 은닉 유닛은 입력층에 연결된 최종 입력 하나만 받는다. 반면 RNN은 은닉 유닛은 두 개의 다른 입력을 받는다. *입력층으로부터 받은 입력*과 같은 은닉층에서 *t-1 타임 스텝의 활성화 출력*을 받는다.   

맨 처음 t=0에서는 은닉 유닛이 0또는 작은 난수로 초기화된다. t>0인 타임 스텝에서는 은닉 유닛이 현재 타임 스텝의 데이터 포인트 $x^{(t)}$와 이전 타임 스텝 t-1의 은닉 유닛 값 $h^{(t-1)}$을 입력으로 받는다.  

## RNN의 활성화 출력 계산

위의 그림에서 유향 에지(directed edge)(층 사이 연결과 순환 연결)는 가중치 행렬과 연관된다. 이 가중치는 특정 시간 t에 종속적이지 않고 전체 시간 축에 공유된다. 단일층 RNN의 각 가중치는 다음과 같다.  

- $W_{xh}$: 입력 $x^{(t)}$와 은닉층 $h$ 사이의 가중치 행렬  
- $W_{hh}$: 순환 에지에 연관된 가중치 행렬  
- $W_{ho}$: 은닉층과 출력층 사이의 가중치 행렬

In [6]:
Image(url='https://git.io/JLdVC', width=700)

구현에 따라 가중치 행렬 $W_{xh}$와 $W_{hh}$를 합쳐 연결된 행렬 $W_{h}=[W_{xh};W_{hh}]$를 사용한다. 

활성화 출력의 계산은 기본적인 다층 퍼셉트론이나 다른 피드포워드 신경망과 매우 비슷하다. 은닉층의 최종 입력 $z_{h}$(활성화 함수를 통과하기 전의 값)는 선형 조합으로 계산된다. 즉, 가중치 행렬과 대응되는 벡터를 곱해서 더한 후 절편 유닛을 더한다.

In [7]:
Image(url='https://git.io/JLdVW', width=700)

위의 그림은 활성화 출력을 계산하는 과정이다.

## 은닉 순환과 출력 순환

지금까지 은닉층에 순환 성질이 있는 순환 신경망을 보았다. 하지만 출력층에서 오는 순환 연결을 가진 모델도 있다. 이런 경우에 이전 타임 스텝의 출력층에서 오는 활성화 $o^{t-1}$을 추가하는 방법은 다음 둘 중 하나이다.  
- 현재 타임 스텝에서 은닉층 $h^{t}$에 추가한다.  
- 현재 타임 스텝에서 출력층 $o^{t}$에 추가한다.  

In [8]:
Image(url='https://git.io/JLdV8', width=700)

실제로 어떻게 동작하는지 보기 위해 이 순환 타입 중 하나의 정방향 계산을 수동으로 수행해 보자. 텐서플로 케라스 API의 `SimpleRNN` 클래스로 출력-출력 순환과 비슷한 순환 층을 정의할 수 있다.  

In [9]:
import tensorflow as tf
tf.random.set_seed(1)

rnn_layer = tf.keras.layers.SimpleRNN(
    units=2, use_bias=True, 
    return_sequences=True)
rnn_layer.build(input_shape=(None, None, 5))

w_xh, w_oo, b_h = rnn_layer.weights

print('W_xh 크기:', w_xh.shape)
print('W_oo 크기:', w_oo.shape)
print('b_h 크기:', b_h.shape)

W_xh 크기: (5, 2)
W_oo 크기: (2, 2)
b_h 크기: (2,)


이 층의 입력 크기는 (None, None, 5)이다.  
첫 번째 차원은 배치 차원(가변적인 배치 크기를 위해 None으로 지정)이고 두 번째 차원은 시퀀스에 해당한다.(가변적인 시퀀스 길이를 위해 None으로 지정) 마지막 차원은 특성에 해당한다. `return_sequence=True`로 지정했으므로 길이가 3인 시퀀스를 입력하면 출력 시퀀스 $<o^{(0)}, o^{(1)}, o^{(2)}>$가 나온다. 그렇지 않으면 최종 출력 $o^{(2)}$만 반환된다.

In [10]:
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5],
    dtype=tf.float32)


## SimepleRNN의 출력:
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))

## 수동으로 출력 계산하기:
out_man = []
for t in range(len(x_seq)):
    xt = tf.reshape(x_seq[t], (1, 5))
    print('타임 스텝 {} =>'.format(t))
    print('   입력           :', xt.numpy())
    
    ht = tf.matmul(xt, w_xh) + b_h    
    print('   은닉           :', ht.numpy())
    
    if t>0:
        prev_o = out_man[t-1]
    else:
        prev_o = tf.zeros(shape=(ht.shape))
        
    ot = ht + tf.matmul(prev_o, w_oo)
    ot = tf.math.tanh(ot)
    out_man.append(ot)
    print('   출력 (수동)     :', ot.numpy())
    print('   SimpleRNN 출력 :'.format(t), output[0][t].numpy())
    print()

타임 스텝 0 =>
   입력           : [[1. 1. 1. 1. 1.]]
   은닉           : [[0.41464037 0.96012145]]
   출력 (수동)     : [[0.39240566 0.74433106]]
   SimpleRNN 출력 : [0.39240566 0.74433106]

타임 스텝 1 =>
   입력           : [[2. 2. 2. 2. 2.]]
   은닉           : [[0.82928073 1.9202429 ]]
   출력 (수동)     : [[0.80116504 0.99129474]]
   SimpleRNN 출력 : [0.80116504 0.99129474]

타임 스텝 2 =>
   입력           : [[3. 3. 3. 3. 3.]]
   은닉           : [[1.243921  2.8803642]]
   출력 (수동)     : [[0.95468265 0.99930704]]
   SimpleRNN 출력 : [0.95468265 0.99930704]



수동으로 정방향 계산을 할 때 하이퍼볼릭 탄젠트(tanh) 활성화 함수를 사용했다. `SimpleRNN`에서 이 함수를 사용하기 때문이다.  
출력 결과에서 볼 수 있듯이 수동으로 계산한 것과 `SimpleRNN` 층의 각 타임 스텝 출력이 정확히 동일하다.

## 긴 시퀀스 학습의 어려움

BPTT(BackPropagation Through Time)는 새로운 문제를 야기시킨다. 손실 함수의 그레이디언트를 계산할 때 곱셈 항${\partial h^{(t)} \over \partial h^{(k)}}$ 때문에 소위 **그레이디언트 폭주**(exploding gradient) 또는 **그레이디언트 소실**(vanishing gradient) 문제가 발생한다. 

In [11]:
Image(url='https://git.io/JLdV4', width=700)

기본적으로 ${\partial h^{(t)} \over \partial h^{(k)}}$는 t-k개의 곱셈으로 이루어진다. 즉, 가중치 w가 t-k번 곱해져 $w^{t-k}$가 된다.  
결국 $|w|<1$이면 t-k가 클 때 이 항이 매우 작아진다. 반면 순환 에지의 가중치 값이 $|w|>1$이면 t-k가 클 때 $w^{t-k}$가 매우 커진다. t-k값이 크다는 것은 긴 시간 의존성을 가진다는 의미이다.  
그레이디언트 소실이나 폭주를 피하는 단순한 방법은 $|w|=1$이 되도록 만드는 것이다.  

실전에서 이 문제에 대한 세 가지 해결책은 다음과 같다.  
- 그레이디언트 클리핑
- TBPTT(Truncated BackPropagation Through Time)
- LSTM(Long Short-Term Memory)  

## LSTM 셀

LSTM은 그레이디언트 소실 문제를 극복하기 위해 처음 소개되었다. LSTM의 기본 구성 요소는 일반 RNN의 은닉층을 표현 또는 대체하는 **메모리 셀**(memory cell)이다.  

그레이디언트 소실과 폭주 문제를 극복하기 위해 각 메모리 셀에 적절한 가중치 w=1을 유지하는 순환 에지가 있다. 이 순환 에지의 출력을 **셀 상태**(cell state)라고 한다. 

In [12]:
Image(url='https://git.io/JLdVR', width=700)

이전 타임 스텝의 셀 상태 $C^{(t-1)}$은 어떤 가중치와도 직접 곱해지지 않고 변경되어 현재 타임 스텝의 셀 상태 $C^{(t)}$을 얻는다.  

메모리 셀의 정보 흐름은 다음에 기술된 몇 개의 연산 유닛(또는 게이트)으로 제어된다.  
그림에서 $\odot$는 원소별 곱셈(element-wise multiplication)을 의미하고, $\oplus$는 원소별 덧셈(element-wise addition)을 나타낸다.  
또 $x^{(t)}$은 타임 스텝 t에서 입력 데이터이고, $h^{(t-1)}$은 타임 스텝 t-1에서 은닉 유닛의 출력 이다.  
네 개의 상자는 시그모이드 함수($\sigma$)나 하이퍼볼릭 탄젠트(tanh) 활성화 함수와 일련의 가중치로 표시된다. 이 상자는 입력에 대해 행렬-벡터 곱셈을 수행한 후 선형 조합된다.   

LSTM 셀에는 **삭제 게이트**(forget gate), **입력 게이트**(input gate), **출력 게이트**(output gate)가 있다. 

# 텐서플로로 시퀀스 모델링을 위한 RNN 구현

## 첫 번째 프로젝트: IMDb 영화 리뷰의 감성 분석

감성 분석을 위해 다대일(many-to-one) 구조의 다층 RNN을 구현해보자.  

#### 영화 리뷰 데이터 준비
8장의 전처리 단계에서 만든 정제된 데이터셋인 movie_data.csv 파일을 다시 사용.

In [13]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

In [14]:
# 코랩에서 실행하는 경우 다음 코드를 실행하세요.
!mkdir ../ch08
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz -O ../ch08/movie_data.csv.gz

--2022-11-24 10:41:49--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz [following]
--2022-11-24 10:41:49--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26521894 (25M) [application/octet-stream]
Saving to: ‘../ch08/movie_data.csv.gz’

../ch08/movie_data. 100%[===================>]  25.29M  --.

In [15]:
import os
import gzip
import shutil


with gzip.open('../ch08/movie_data.csv.gz', 'rb') as f_in, open('movie_data.csv', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

In [16]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')

df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


In [17]:
# 단계 1: 데이터셋 만들기
target = df.pop('sentiment')

ds_raw = tf.data.Dataset.from_tensor_slices(
    (df.values, target.values))

## 확인:
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


In [18]:
## 훈련/검증/테스트 분할
tf.random.set_seed(1)

ds_raw = ds_raw.shuffle(
    50000, reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

In [19]:
## 단계 2: 고유 토큰(단어) 찾기
from collections import Counter

tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()

for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0])
    token_counts.update(tokens)
    
print('어휘 사전 크기:', len(token_counts))

어휘 사전 크기: 87007


In [20]:
## 단계 3: 고유 토큰을 정수로 인코딩하기
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)

example_str = 'This is an example!'
encoder.encode(example_str)

[232, 9, 270, 1123]

검증 데이터와 테스트 데이터에 있는 토큰이 훈련 데이터에 없다면 매핑되지 않을 수 있다.  
q개의 토큰이 있고 이전에 본 적이 없으며 `token_counts`에 포함되지 않은 모든 토큰은 정수 q+1에 할당된다.  
다른 말로 하면 인덱스 q+1이 알려지지 않은 단어를 위해 예약된다. 예약된 또 다른 값은 정수 0이다. 시퀀스 길이를 조절하기 위한 용도로 사용한다.  

데이터셋 객체의 `map()` 메서드를 사용하여 다른 변환을 적용하듯이 데이터셋에 있는 각 텍스트를 변환할 수 있다. 하지만 여기에는 작은 문제가 있다. 텍스트 데이터가 텐서 객체에 들어가 있어 즉시 실행 모드에서 텐서의 `numpy()` 메서드로 참조할 수 있다. 하지만 `map()` 메서드로 변환하는 동안에는 즉시 실행이 비활성화된다. 이 문제를 해결하기 위해 두 개의 함수를 정의한다. 첫 번째 함수는 즉시 실행 모드가 활성화된 것처럼 입력 텐서를 다룬다.

In [21]:
## 단계 3-A: 변환 함수 정의하기
def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    return encoded_text, label

두 번째 함수에서 첫 번째 함수를 `tf.py_function` 으로 감싸서 `map()` 메서드에서 사용할 수 있는 텐서플로 연산으로 변환한다. 

In [22]:
## 단계 3-B: 인코딩 함수를 텐서플로 연산으로 감싸기
def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label], 
                          Tout=(tf.int64, tf.int64))

In [23]:
ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
    print('시퀀스 길이:', example[0].shape)
    
example

시퀀스 길이: (24,)
시퀀스 길이: (179,)
시퀀스 길이: (262,)
시퀀스 길이: (535,)
시퀀스 길이: (130,)


(<tf.Tensor: shape=(130,), dtype=int64, numpy=
 array([  579,  1296,    32,   425,    40,   763,  9267,    65,   280,
          308,     6,   481,   155,   473,     2,     3,   684,     9,
          781,   176,   959,   730,  3917,    67,  9905,    13,   277,
           24,    35,   371, 16368,     6,    14, 17231,    29,   187,
         1651,   489,   503,   480,   143,    32,   270,  5851,  2402,
           13,  3592,  3443,   425,  3313,   256,   257,  1577,   117,
            8,   698,   270,   564,    56,     8,    42,  7517,  2629,
          820,    25,    60,    79,   343,    32,   645,    14,   528,
          241,    32,  1980,     8,    56,     8,    42,  1364,   573,
         5183,    43,    12,  3870,    32,   312,   642,   251,  1401,
        17232,     8,   698,   257,   750,     2,     9,    76,   235,
            8,    42,   235,   840,   666,   258, 17233,   419,    32,
        17234,   585,   420,   840,    25,    40,    13,    14,   198,
          266,   623,   173,  

지금까지 단어 시퀀스를 정수 시퀀스로 변환했다. 하지만 여전히 해결할 문제가 하나 있다. 시퀀스 길이가 다르다.  
일반적으로 RNN은 다른 길이의 시퀸스를 다룰 수 있지만 미니 배치에 있는 시퀀스는 효율적으로 텐서에 저장하기 위해 동일한 길이가 되어야 한다.  

크기가 다른 원소를 가진 데이터셋을 미니 배치로 나누기 위해 텐서플로는 `padded_batch()` 메서드를 제공한다. 이 메서드는 하나의 배치에 포함되는 모든 원소를 자동으로 0으로 패딩하여 배치에 있는 모든 시퀀스가 동일한 길이가 되도록 만든다. 

In [24]:
## 일부 데이터 추출하기
ds_subset = ds_train.take(8)
for example in ds_subset:
    print('개별 샘플 크기:', example[0].shape)

## 배치 데이터 만들기
ds_batched = ds_subset.padded_batch(
    4, padded_shapes=([-1], []))

for batch in ds_batched:
    print('배치 차원:', batch[0].shape)

개별 샘플 크기: (119,)
개별 샘플 크기: (688,)
개별 샘플 크기: (308,)
개별 샘플 크기: (204,)
개별 샘플 크기: (326,)
개별 샘플 크기: (240,)
개별 샘플 크기: (127,)
개별 샘플 크기: (453,)
배치 차원: (4, 688)
배치 차원: (4, 453)


출력된 텐서 크기에서 알 수 있듯이 첫 번째 배치의 열 개수(즉, .shape[1])는 688이다. 처음 네 개의 샘플이 하나의 배치가 되었고 이 샘플 중에서 가장 큰 크기를 사용했다.  
다시 말하면 이 배치에 있는 세 개의 다른 샘플을 이 크기에 맞도록 필요한 만큼 패딩을 추가했다. 비슷하게 두 번째 배치의 열 크기는 다음 네 개의 샘플 중 가장 큰 크기인 453이다. 여기서도 최대 길이보다 작은 다른 샘플에 패딩을 추가했다.  

세 개의 데이터셋을 모두 배치 크기 32의 미니 배치로 나눈다.

In [25]:
## 배치 데이터 만들기
train_data = ds_train.padded_batch(
    32, padded_shapes=([-1],[]))

valid_data = ds_valid.padded_batch(
    32, padded_shapes=([-1],[]))

test_data = ds_test.padded_batch(
    32, padded_shapes=([-1],[]))

이제 데이터가 이어지는 절에서 구현할 RNN 모델에 적합한 포맷이 되었다. 

#### 문장 인코딩을 위한 임베딩 층 
이전의 데이터 준비 단계에서 동일한 길이의 시퀀스르 생성했다. 이 시퀀스의 원소는 고유한 단어의 인덱스에 해당하는 정수이다. 이런 단어 인덱스를 입력 특성으로 변환하는 몇 가지 방법이 있다.  
간단하게 원-핫 인코딩을 적용하여 인덱스를 0 또는 1로 이루어진 벡터로 변환할 수 있다. 각 단어는 전체 데이터셋의 고유한 단어의 수에 해당하는 크기를 가진 벡터로 변환된다.  
고유한 단어의 수(어휘 사전의 크기)가 $10^{4}-10^5$ 단위가 될 수 있으며 입력 특성의 개수도 마찬가지이다. 이렇 게 많은 특성에서 훈련된 모델은 **차원의 저주**(curse of dimensionality)로 인한 영향을 받는다. 또 하나를 제외하고 모든 원소가 0이므로 특성 벡터가 매우 희소해진다.  

좀 더 고급스러운 방법은 각 단어를 실수 값을 가진 고정된 길이의 벡터로 변환하는 것이다. 원-핫 인코딩과 달리 고정된 길이의 벡터를 사용하여 무한히 많은 실수를 표현할 수 있다.  

**임베딩**(embedding)이라고 하는 특성 학습 기법을 사용하여 데이터셋에 있는 단어를 표현하는 데 중요한 특성을 자동으로 학습할 수 있다. 고유한 단어의 수를 $x_{words}$라고 하면 고유한 단어의 수보다 훨씬 작게 임베딩 벡터(또는 임베딩 차원) 크기를 선택하여 전체 어휘를 특성으로 나타낸다.  

원-핫 인코딩에 비해 임베딩의 장점은 다음과 같다.  
- 특성 공간의 차원이 축소되므로 차원의 저주로 인한 영향을 감소시킨다.  
- 신경망에서 임베딩 층이 최적화되기 때문에 중요한 특성이 추출된다.

In [26]:
Image(url='https://git.io/JLdV0', width=700)

n+2 크기(토큰 개수 n에 패딩을 위해 예약된 인덱스 0과 토큰 집합에 없는 단어를 위해 예약된 인덱스 n+1이 추가)의 토큰 집합이 주어지면 $(n+2)\times embedding\_dim$크기의 임베딩 행렬이 만들어진다. 이 행렬의 행은 토큰에 연관된 수치 특성을 표현한다. 

In [27]:
from tensorflow.keras.layers import Embedding


model = tf.keras.Sequential()

model.add(Embedding(input_dim=100,
                    output_dim=6,
                    input_length=20,
                    name='embed-layer'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, 20, 6)             600       
                                                                 
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


이 모델의 입력(임베딩 층)은 $batchsize \times input\_length$ 차원을 가진 랭크 2여야 한다. 여기서 $input\_length$는 시퀀스 길이이다.  
출력 차원은 $batchsize \times input\_length \times embedding\_dim$이다. $embedding\_dim$은 임베딩 특성의 크기이다. 


#### RNN 모델 만들기  
순환 층에는 다음과 같은 클래스를 사용할 수 있다.  
- SimpleRNN: 완전 연결 순환 층인 기본 RNN  
- LSTM: 긴 의존성을 감지할 수 있는 LSTM RNN  
- GRU: LTSM의 대안인 GRU 유닛을 사용한 순환 층

In [28]:
## SimpleRNN 층으로 RNN 모델 만들기
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(1000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 36,193
Trainable params: 36,193
Non-trainable params: 0
_________________________________________________________________


In [29]:
## LSTM 층으로 RNN 모델 만들기
from tensorflow.keras.layers import LSTM


model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          320000    
                                                                 
 lstm (LSTM)                 (None, None, 32)          8320      
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 336,673
Trainable params: 336,673
Non-trainable params: 0
_________________________________________________________________


In [30]:
## GRU 층으로 RNN 모델 만들기
from tensorflow.keras.layers import GRU

model = Sequential()
model.add(Embedding(10000, 32))
model.add(GRU(32, return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          320000    
                                                                 
 gru (GRU)                   (None, None, 32)          6336      
                                                                 
 gru_1 (GRU)                 (None, 32)                6336      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 332,705
Trainable params: 332,705
Non-trainable params: 0
_________________________________________________________________


#### 감성 분석 작업을 위한 RNN 모델 만들기
시퀀스 길이가 길기 때문에 길게 미치는 영향을 감지하기 위해 LSTM 층을 사용한다. 또한, `Bidirectional` 클래스로 LSTM 층을 감싼다.  
이 층은 입력 시퀀스를 처음부터 끝까지 그리고 끝에서 처음까지 양방향으로 순환 층을 통과하도록 한다. 

In [31]:
embedding_dim = 20
vocab_size = len(token_counts) + 2

tf.random.set_seed(1)

## 모델 생성
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        name='embed-layer'),
    
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, name='lstm-layer'),
        name='bidir-lstm'), 

    tf.keras.layers.Dense(64, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

## 컴파일과 훈련:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = bi_lstm_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

## 테스트 데이터에서 평가
test_results= bi_lstm_model.evaluate(test_data)
print('테스트 정확도: {:.2f}%'.format(test_results[1]*100))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1740180   
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,792,021
Trainable params: 1,792,021
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 62s 90ms/step - loss: 0.4990 - accuracy: 0.7466 - val_loss: 0.3863 - val_accuracy: 0.8594
Epoch 2/10
625/625 [============

In [32]:
if not os.path.exists('models'):
    os.mkdir('models')


bi_lstm_model.save('models/Bidir-LSTM-full-length-seq.h5')

SimpleRNN과 같은 다른 종류의 순환 층을 사용할 수도 있다.  
일반적인 순환 층으로 만든 모델은 좋은 예측 성능을 달성하지 못한다. 예를 들어 이전 코드에서 양방향 LSTM 층을 단방향 SimpleRNN 층으로 바꾸고 전체 길이를 사용한 시퀀스로 모델을 훈련하면 훈련하는 동안 손실이 전혀 감소하지 않는다. 이 데이터셋에 있는 시퀀스가 너무 길기 때문이다. SimpleRNN 층을 사용한 모델은 장기간 의존성을 학습할 수 없고 그레이디언트 감소나 폭주로 인한 영향을 받는다.  

SimpleRNN으로 이 데이터셋에서 납득할 수 있는 수준의 예측 성능을 얻으려면 시퀀스 길이를 줄여야 한다.  
영화 리뷰의 마지막 문장에 감성에 관한 정보가 많이 담겨 있다고 가정할 수 있다. 따라서 각 리뷰의 마지막 부분에만 초점을 맞추어 보자. `preprocess_datasets()` 라는 헬펌 함수를 만들어 전처리 단계 2~4를 연결한다.  
이 함수의 매개변수는 각 리뷰에서 얼마나 많은 토큰을 사용할지 결정하는 `max_seq_length`이다. 예를 들어 `max_seq_length=None`으로 지정하면 이전처럼 전체 길이의 시퀀스가 사용된다. 

#### 짧은 시퀀스에 SimpleRNN 적용하기

In [33]:
def preprocess_datasets(
    ds_raw_train, 
    ds_raw_valid, 
    ds_raw_test,
    max_seq_length=None,
    batch_size=32):
    
    ## 단계 1: (데이터셋 만들기 이미 완료)
    ## 단계 2: 고유 토큰 찾기
    tokenizer = tfds.deprecated.text.Tokenizer()
    token_counts = Counter()

    for example in ds_raw_train:
        tokens = tokenizer.tokenize(example[0].numpy()[0])
        if max_seq_length is not None:
            tokens = tokens[-max_seq_length:]
        token_counts.update(tokens)

    print('어휘 사전 크기:', len(token_counts))


    ## 단계 3: 텍스트 인코딩하기
    encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
    def encode(text_tensor, label):
        text = text_tensor.numpy()[0]
        encoded_text = encoder.encode(text)
        if max_seq_length is not None:
            encoded_text = encoded_text[-max_seq_length:]
        return encoded_text, label

    def encode_map_fn(text, label):
        return tf.py_function(encode, inp=[text, label], 
                              Tout=(tf.int64, tf.int64))

    ds_train = ds_raw_train.map(encode_map_fn)
    ds_valid = ds_raw_valid.map(encode_map_fn)
    ds_test = ds_raw_test.map(encode_map_fn)

    ## 단계 4: 배치 데이터 만들기
    train_data = ds_train.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    valid_data = ds_valid.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    test_data = ds_test.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    return (train_data, valid_data, 
            test_data, len(token_counts))

In [34]:
def build_rnn_model(embedding_dim, vocab_size,
                    recurrent_type='SimpleRNN',
                    n_recurrent_units=64,
                    n_recurrent_layers=1,
                    bidirectional=True):

    tf.random.set_seed(1)

    # 모델 생성
    model = tf.keras.Sequential()
    
    model.add(
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            name='embed-layer')
    )
    
    for i in range(n_recurrent_layers):
        return_sequences = (i < n_recurrent_layers-1)
            
        if recurrent_type == 'SimpleRNN':
            recurrent_layer = SimpleRNN(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='simprnn-layer-{}'.format(i))
        elif recurrent_type == 'LSTM':
            recurrent_layer = LSTM(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='lstm-layer-{}'.format(i))
        elif recurrent_type == 'GRU':
            recurrent_layer = GRU(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='gru-layer-{}'.format(i))
        
        if bidirectional:
            recurrent_layer = Bidirectional(
                recurrent_layer, name='bidir-'+recurrent_layer.name)
            
        model.add(recurrent_layer)

    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

In [35]:
from tensorflow.keras.layers import Bidirectional


batch_size = 32
embedding_dim = 20
max_seq_length = 100

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test, 
    max_seq_length=max_seq_length, 
    batch_size=batch_size
)


vocab_size = n + 2

rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='SimpleRNN', 
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)

rnn_model.summary()

어휘 사전 크기: 58063
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1161300   
                                                                 
 bidir-simprnn-layer-0 (Bidi  (None, 128)              10880     
 rectional)                                                      
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,180,501
Trainable params: 1,180,501
Non-trainable params: 0
_________________________________________________________________


In [36]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])


history = rnn_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

Epoch 1/10
625/625 [==============================] - 147s 231ms/step - loss: 0.7001 - accuracy: 0.5069 - val_loss: 0.6920 - val_accuracy: 0.5194
Epoch 2/10
625/625 [==============================] - 146s 234ms/step - loss: 0.6705 - accuracy: 0.5681 - val_loss: 0.5748 - val_accuracy: 0.7288
Epoch 3/10
625/625 [==============================] - 138s 221ms/step - loss: 0.5379 - accuracy: 0.7278 - val_loss: 0.5316 - val_accuracy: 0.7792
Epoch 4/10
625/625 [==============================] - 140s 224ms/step - loss: 0.3444 - accuracy: 0.8539 - val_loss: 0.4213 - val_accuracy: 0.8198
Epoch 5/10
625/625 [==============================] - 132s 211ms/step - loss: 0.1975 - accuracy: 0.9271 - val_loss: 0.5590 - val_accuracy: 0.7926
Epoch 6/10
625/625 [==============================] - 125s 200ms/step - loss: 0.1276 - accuracy: 0.9553 - val_loss: 0.5759 - val_accuracy: 0.8220
Epoch 7/10
625/625 [==============================] - 130s 208ms/step - loss: 0.0638 - accuracy: 0.9798 - val_loss: 0.7014 -

In [37]:
results = rnn_model.evaluate(test_data)

782/782 [==============================] - 31s 40ms/step - loss: 0.8691 - accuracy: 0.8241


In [38]:
print('테스트 정확도: {:.2f}%'.format(results[1]*100))

테스트 정확도: 82.41%


### 연습 문제:  
#### 전체 길이를 사용한 시퀀스에 단방향 SimpleRNN 적용하기

In [39]:
batch_size = 32
embedding_dim = 20
max_seq_length = None

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test, 
    max_seq_length=max_seq_length, 
    batch_size=batch_size
)


vocab_size = n + 2

rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='SimpleRNN', 
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=False)

rnn_model.summary()

어휘 사전 크기: 87007
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1740180   
                                                                 
 simprnn-layer-0 (SimpleRNN)  (None, 64)               5440      
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,749,845
Trainable params: 1,749,845
Non-trainable params: 0
_________________________________________________________________


In [40]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])

history = rnn_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

Epoch 1/10
625/625 [==============================] - 559s 892ms/step - loss: 0.7000 - accuracy: 0.5041 - val_loss: 0.7149 - val_accuracy: 0.5140
Epoch 2/10
625/625 [==============================] - 549s 879ms/step - loss: 0.7020 - accuracy: 0.4983 - val_loss: 0.7042 - val_accuracy: 0.4916
Epoch 3/10
625/625 [==============================] - 544s 871ms/step - loss: 0.6983 - accuracy: 0.4985 - val_loss: 0.7046 - val_accuracy: 0.4948
Epoch 4/10
625/625 [==============================] - 540s 865ms/step - loss: 0.6967 - accuracy: 0.5034 - val_loss: 0.7011 - val_accuracy: 0.4934
Epoch 5/10
625/625 [==============================] - 543s 870ms/step - loss: 0.6956 - accuracy: 0.5044 - val_loss: 0.6976 - val_accuracy: 0.4994
Epoch 6/10
625/625 [==============================] - 537s 859ms/step - loss: 0.6950 - accuracy: 0.5024 - val_loss: 0.6972 - val_accuracy: 0.4966
Epoch 7/10
625/625 [==============================] - 544s 870ms/step - loss: 0.6944 - accuracy: 0.5008 - val_loss: 0.6955 -

# 부록

#### A--데이터셋을 만드는 다른 방법: tensorflow_datasets 사용하기

In [41]:
imdb_bldr = tfds.builder('imdb_reviews')
print(imdb_bldr.info)

imdb_bldr.download_and_prepare()

datasets = imdb_bldr.as_dataset(shuffle_files=False)

datasets.keys()

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=Unknown size,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
       

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRAOOAL/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRAOOAL/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


dict_keys([Split('train'), Split('test'), Split('unsupervised')])

In [42]:
imdb_train = datasets['train']
imdb_train = datasets['test']

#### B--Tokenizer와 Encoder

In [43]:
vocab_set = {'a', 'b', 'c', 'd'}
encoder = tfds.deprecated.text.TokenTextEncoder(vocab_set)
print(encoder)

print(encoder.encode(b'a b c d, , : .'))

print(encoder.encode(b'a b c d e f g h i z'))

<TokenTextEncoder vocab_size=6>
[2, 3, 4, 1]
[2, 3, 4, 1, 5, 5, 5, 5, 5, 5]


#### C--케라스로 텍스트 전처리하기

In [44]:
TOP_K = 200
MAX_LEN = 10

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=TOP_K)

tokenizer.fit_on_texts(['this is an example', 'je suis en forme '])
sequences = tokenizer.texts_to_sequences(['this is an example', 'je suis en forme '])
print(sequences)

tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_LEN)

[[1, 2, 3, 4], [5, 6, 7, 8]]


array([[0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 5, 6, 7, 8]], dtype=int32)

In [45]:
TOP_K = 20000
MAX_LEN = 500

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=TOP_K)

tokenizer.fit_on_texts(
    [example['text'].numpy().decode('utf-8') 
     for example in imdb_train])

x_train = tokenizer.texts_to_sequences(
    [example['text'].numpy().decode('utf-8')
     for example in imdb_train])

print(len(x_train))


x_train_padded = tf.keras.preprocessing.sequence.pad_sequences(
    x_train, maxlen=MAX_LEN)

print(x_train_padded.shape)

25000
(25000, 500)


#### D--임베딩

In [46]:
from tensorflow.keras.layers import Embedding


tf.random.set_seed(1)
embed = Embedding(input_dim=100, output_dim=4)

inp_arr = np.array([1, 98, 5, 6, 67, 45])
tf.print(embed(inp_arr))
tf.print(embed(inp_arr).shape)

tf.print(embed(np.array([1])))

[[-0.0208060984 0.0142502077 0.0475785471 -0.00649005175]
 [-0.00420691818 -0.0375086069 -0.00477621704 0.00311584398]
 [0.028728161 -0.0440448038 -0.0428906195 -0.019158531]
 [-0.0248817336 0.0408470519 -0.00285203382 -0.0257614851]
 [0.0443614833 0.00331580639 0.043055404 -0.011118304]
 [-0.0281324144 0.00720113516 0.0192188732 -0.0186921246]]
TensorShape([6, 4])
[[-0.0208060984 0.0142502077 0.0475785471 -0.00649005175]]
